<a href="https://colab.research.google.com/github/marianaolmedo/Simulacion-II/blob/main/Muestreo_Importancia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rd
import time as tt

In [ ]:
def f(x):
    return np.cos((x * np.pi) / 2)

In [ ]:
def g(x):
    return (24 / (24 - np.pi ** 2)) * (1 - ((np.pi ** 2) * (x ** 2)) / 8)

**Método de Monte Carlo Crudo**

Si queremos estimar
$$I = \int_0^1 f(x) dx$$
se genera $n$ números aleatorios
$$U_i \sim U(0,1)$$
y usando el estimador
$$\hat{Θ}= \frac{1}{n} \sum_{i=1}^n f(U_i)$$

In [ ]:
def estimacion_crudo(n):
  G = []
  for i in range(n):
    u = rd.random()
    G.append(f(u))
  I = sum(G)/n
  return I

def MC_crudo(n, N):
  start = tt.perf_counter()
  lista = []
  for i in range(N):
    e = estimacion_crudo(n)
    lista.append(e)
  end = tt.perf_counter()
  tiempo = end - start
  return np.mean(lista), np.var(lista, ddof=1), np.std(lista, ddof=1), tiempo

**Muestreo por Importancia**

Si queremos estimar
$$I = \int_0^1 f(x) dx$$
podemos reescribirla multiplicando y dividiendo por una función de densidad $g(x)$
$$I = ∫_0^1 \frac{f(x)}{g(x)} g(x) dx = \mathbb{E}_g \left[\frac{f(x)}{g(x)}\right]$$
donde
$$X \sim g(x)$$
y usando el estimador
$$\hat{Θ}_{IS}= \frac{1}{n} \sum_{i=1}^n \frac{f(X_i)}{g(X_i)}$$

In [9]:
def muestra_g(n):
    c = 24 / (24 - np.pi ** 2)
    muestras = []
    while len(muestras) < n:
        u_2 = rd.random()
        w_2 = rd.uniform(0, c)
        if w_2 <= g(u_2):
            muestras.append(u_2)
    return np.array(muestras)

In [10]:
def estimacion_importancia(n):
  G_2 = []
  u_3 = muestra_g(n)
  for i in u_3:
    G_2.append(f(i)/g(i))
  I_2 = sum(G_2)/n
  return I_2

In [11]:
def MC_importancia(n, N):
    start = tt.perf_counter()
    lista2 = []
    for i in range(N):
        e_2 = estimacion_importancia(n)
        lista2.append(e_2)
    end = tt.perf_counter()
    tiempo2 = end - start
    return np.mean(lista2), np.var(lista2, ddof=1), np.std(lista2, ddof=1), tiempo2

**Comparación de ambos métodos**

In [12]:
n = 200     # Cada simulación se hace con 200 puntos
N = 40      # Hace 40 simulaciones

resultado_crudo = MC_crudo(n, N)
resultado_mi= MC_importancia(n, N)

reduccion = (1 - resultado_mi[1] / resultado_crudo[1]) * 100

print("=== Resultados MC Crudo ===")
print("Estimación promedio:", resultado_crudo[0])
print("Varianza:", resultado_crudo[1])
print("Desviación estándar:", resultado_crudo[2])
print("Tiempo:", resultado_crudo[3])

print("\n=== Resultados Muestreo por Importancia ===")
print("Estimación promedio:", resultado_mi[0])
print("Varianza:", resultado_mi[1])
print("Desviación estándar:", resultado_mi[2])
print("Tiempo:", resultado_mi[3])

print(f"\nReducción porcentual de la varianza: {reduccion:.2f}%")


=== Resultados MC Crudo ===
Estimación promedio: 0.6330037626656436
Varianza: 0.0004897905744542134
Desviación estándar: 0.02213121267473189
Tiempo: 0.01054835899998352

=== Resultados Muestreo por Importancia ===
Estimación promedio: 0.6155471817552516
Varianza: 5.976314141761802e-05
Desviación estándar: 0.0077306624177762425
Tiempo: 0.035992537000026914

Reducción porcentual de la varianza: 87.80%
